In [1]:
from adversary.core.noise_generators import AdditiveNoiseGenerator
from adversary.attacks.whitebox.wb_targeted_attack import TargetedWhiteBoxAttack
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

from adversary.core.tensor_ops import TensorOps

In [2]:
gener = AdditiveNoiseGenerator(framework="tf", epsilon=0.2, dist='uniform')

tensor = gener.generate(shape=(1080, 1080))
print(np.max(tensor.numpy()))

tensor_np = tensor.numpy()
# plt.figure(figsize=(10, 10))
# plt.imshow(tensor_np, cmap='Greys', interpolation='none', vmin=0, vmax=1)
# plt.colorbar()
# plt.title('Normalized Tensor [1080, 1080]')
# plt.axis('off')
# plt.show()

0.19999985


In [6]:
type(gener)

adversary.core.noise_generators.additive_noise_generator.AdditiveNoiseGeneratorTF

In [3]:
model = InceptionV3(weights='imagenet')

In [4]:
attacker = TargetedWhiteBoxAttack(model, "cce", "adam")

In [5]:
# attacker.model_info["output_shape"][1]
attacker.attack(None, 5)

TypeError: AdditiveNoiseGeneratorTF.generate() missing 1 required positional argument: 'shape'